# Búsqueda hiperparámteros del modelo SVM 



- Cada bloque de 1000 días tiene su propia dinámica, volatilidad y características del mercado. Los hiperparámetros óptimos pueden variar de bloque a bloque.

- Sin embargo si pretendemos calcular hiperparámetros de cada bloque y de cada acción, los tiempos de cómputo se disparan.

- Por ello realizaremos búsqueda de hiperparámetros usando un bloque y una acción determinados. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

# Escalamiento
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Métricas
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error, r2_score

# modelo SVR
from sklearn.svm import SVR
from sklearn.svm import LinearSVR
from sklearn.model_selection import TimeSeriesSplit

# hiperparametros
from sklearn.model_selection import GridSearchCV


## Lectura de datasets 

In [2]:
# Nombre de las acciones
tickers = [
    'TSCO', 'AZN', 'BARC', 'BP', 'BATS', 'HLMA',
    'HSBA', 'JMAT', 'LGEN', 'MKS', 'PSON', 'REL',
    'NWG', 'SHEL', 'SGE', 'SBRY', 'SDR', 'SVT',
    'SMIN', 'SSE', 'VOD'
]

data_frames = {} # Inicializo diciconario de data frames

for ticker in tickers: 
    
    df = pd.read_csv(f'datasets_features/{ticker}.csv')  # leo el dataset
    data_frames[ticker] = df                             # guardo dataframes
    
print(data_frames.keys())

dict_keys(['TSCO', 'AZN', 'BARC', 'BP', 'BATS', 'HLMA', 'HSBA', 'JMAT', 'LGEN', 'MKS', 'PSON', 'REL', 'NWG', 'SHEL', 'SGE', 'SBRY', 'SDR', 'SVT', 'SMIN', 'SSE', 'VOD'])


## Función que calcula hiperparametros (1 acción - 1 bloque)

In [7]:
def hiperparametros_SVM(df_bloque):
    '''
    Dado un bloque de dataframe (1000 días, o todo el dataset) calcula los hiperámetros óptimos 
    ''' 
    # 1) ------------- Partición de datos ----------------
    
    # Índice de separación
    N = len(df_bloque)
    split_idx = int(N * 0.75) 
    
    df_features = df_bloque.drop(columns=['Date', 'Open', 'High', 'Low','Close','Volume', 'prev_close', 'target']) # todas columnas r_i
    target      = df_bloque['target']  # columna del target

    
    X_train = df_features.iloc[ : split_idx]
    X_test  = df_features.iloc[split_idx : ]
    y_train = target.iloc[ : split_idx]
    y_test  = target.iloc[split_idx : ]
    
    #print(split_idx)
    #display(X_train.head())
    display(X_train.shape)
    display(X_test.shape)

    # 2) ------------ Estandarización ---------------------
    # Features 
    scaler_x = MinMaxScaler(feature_range=(-1,1))
    X_train_s = scaler_x.fit_transform(X_train)     
    X_test_s  = scaler_x.transform(X_test)

    # Target (al estandarizar son 2D)
    scaler_y = MinMaxScaler(feature_range=(-1,1))
    y_train_s = scaler_y.fit_transform(y_train.values.reshape(-1, 1)).ravel()    # Necesario reshape, scaler espera 2D      
    y_test_s  = scaler_y.transform(y_test.values.reshape(-1, 1))


    # 3) ------------- Busco hiperparametros --------------------
    # Grid 
    param_grid = {
        'kernel': ['linear', 'rbf'],
        'C': [0.1, 1, 10, 100],
        'gamma': ['scale', 'auto', 0.01, 0.001],
        'epsilon': [0.01, 0.1, 0.2]
    }

    # Búsqueda por grid search con validación cruzada

    grid = GridSearchCV(
        estimator = SVR(),
        param_grid = param_grid,
        scoring = 'neg_mean_squared_error', # 'neg_mean_squared_error' devuelve el negativo del MSE → cuanto más alto (menos negativo), mejor (menos error)
        n_jobs = -1,
        cv = TimeSeriesSplit(n_splits=5),
        refit = True,
        verbose = 0,
        return_train_score = False #defecto false
    )

    grid.fit(X=X_train_s, y=y_train_s)
    
    #medias_test     = grid.cv_results_['mean_test_score']  #array
    #std_test        = grid.cv_results_['std_test_score']   #array
    #posibles_params = grid.cv_results_['params']           #lista de diccionarios

    # Almaceno resultado
    resultado = grid.best_params_.copy()
    resultado['best_score'] = grid.best_score_  #añado una clave más
    
    return resultado
    
    

### Ejemplo búsqueda hiperparámetros en un bloque  

In [5]:
# Elijo un data set
df = data_frames['AZN']

# Calculo hiperparámetros (1 bloque)
hiper_1bloque = hiperparametros_SVM(df[0:1000])

(750, 20)

(250, 20)

In [8]:
# Elijo un data set
df = data_frames['AZN']

# Calculo hiperparámetros (1 bloque)
hiper_1bloque_s = hiperparametros_SVM(df[0:1000])

(750, 20)

(250, 20)

In [9]:
display(hiper_1bloque)     # grid.fit(X=X_train, y=y_train)
display(hiper_1bloque_s)   # grid.fit(X=X_train_s, y=y_train_s)

{'C': 0.1,
 'epsilon': 0.01,
 'gamma': 'auto',
 'kernel': 'rbf',
 'best_score': -0.00013280459288084586}

{'C': 1,
 'epsilon': 0.2,
 'gamma': 0.001,
 'kernel': 'rbf',
 'best_score': -0.03926488641586433}

**Nota:** La segunda opción me da mejores métricas